7.25 Constrained maximum likelihood estimate of mean and covariance

Change of variable $\Sigma^{-1}$ and $\Sigma^{-1}\mu$ 

Note that matrix inversion is a convex operation

In [1]:
import numpy as np
import cvxpy as cp
from matplotlib import pyplot as plt
from scipy.linalg import sqrtm

In [89]:
n = 20 # Dimension of input
#np.random.seed(0) # Set random variable seed

# Generate distribution with true mean mu
# and true covariance Sigma such that Sigma^(-1)mu >= 0

Sigma = np.random.rand(n, n)
Sigma = 0.5 * (Sigma + Sigma.T)
Sigma = Sigma + n * np.eye(n)
temp_mu = np.random.rand(n, 1)
res = np.linalg.solve(Sigma, temp_mu)
res = np.maximum(res, 0)
mu = Sigma.dot(res).flatten()

# Draw N random samples from distribution
# N = 25
N = 500
X = np.random.multivariate_normal(mu, Sigma, N).T


#X = np.random.randn(n, N)
#MU = np.zeros((n, N))
#for i in range(N):
#    MU[:, i] = mu.flatten()
#Sq = sqrtm(Sigma)
#X = Sq.dot(X)
#X = X + MU

# n: dimension of vector
# N: number of samples from distribution
# X: matrix containing  N samples from distribution (n x N)

In [90]:
Omega = cp.Variable((n,n), PSD=True) # The precision matrix, Ω, is defined to be the inverse of the covariance matrix
h = cp.Variable(n, nonneg=True) # h = \Sigma^{-1}\mu
objective = cp.Maximize(
    N * cp.log_det(Omega) 
    - cp.sum([cp.quad_form(x, Omega) for x in X.T])
    + 2 * cp.sum(X.T @ h)
    - N * cp.matrix_frac(h, Omega) # h^T Omega^{-1} h
)
constraints = []
prob = cp.Problem(objective, constraints)
result = prob.solve()

WARNING - large complementary slackness residual: 2.032198


In [91]:
Sigma_est = np.linalg.inv(Omega.value)
mu_est = Sigma_est @ h.value
print('l2 distance between true mean and estimated mean:', np.linalg.norm(mu - mu_est, 2))
print('Frobenius norm between true cov and estimated cov:', np.linalg.norm(Sigma - Sigma_est, 'fro'))

l2 distance between true mean and estimated mean: 0.7770216241238542
Frobenius norm between true cov and estimated cov: 18.413828955023643


In [92]:
mu_emp = np.mean(X, axis=1)
print('l2 distance between true mean and empirical mean:', np.linalg.norm(mu - mu_emp, 2))
Sigma_emp = (X.T - mu_emp).T @ (X.T - mu_emp) / N
print('Frobenius norm between true cov and empirical cov:', np.linalg.norm(Sigma - Sigma_emp, 'fro'))

l2 distance between true mean and empirical mean: 0.8682822402125457
Frobenius norm between true cov and empirical cov: 18.47092641169903


In [93]:
# without the constraint that \Sigma^{-1}\mu >= 0 
Omega = cp.Variable((n,n), PSD=True) # The precision matrix, Ω, is defined to be the inverse of the covariance matrix
h = cp.Variable(n) # h = \Sigma^{-1}\mu
objective = cp.Maximize(
    N * cp.log_det(Omega) 
    - cp.sum([cp.quad_form(x, Omega) for x in X.T])
    + 2 * cp.sum(X.T @ h)
    - N * cp.matrix_frac(h, Omega)
)
constraints = []
prob = cp.Problem(objective, constraints)
result = prob.solve()

WARNING - large complementary slackness residual: 3.669426


In [94]:
# verify the convex optimization solution agrees with the analytical solution
print(np.linalg.norm(np.linalg.inv(Omega.value) @ h.value - mu_emp, 2))
print(np.linalg.norm(np.linalg.inv(Omega.value) - Sigma_emp, 'fro'))

0.004200840206031278
9.05651767063618e-06
